In [1]:
import os
from dotenv import load_dotenv
import openai
from langchain import OpenAI

In [3]:
# Azure
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_DEPLOYMENT_ENDPOINT = os.getenv("OPENAI_DEPLOYMENT_ENDPOINT")
OPENAI_DEPLOYMENT_NAME = os.getenv("OPENAI_DEPLOYMENT_NAME")
OPENAI_MODEL_NAME = os.getenv("OPENAI_MODEL_NAME")
OPENAI_API_VERSION = os.getenv("OPENAI_API_VERSION")
OPENAI_EMBEDDING_DEPLOYMENT_NAME = os.getenv("OPENAI_EMBEDDING_DEPLOYMENT_NAME")
OPENAI_EMBEDDING_MODEL_NAME = os.getenv("OPENAI_EMBEDDING_MODEL_NAME")
OPENAI_DEPLOYMENT_VERSION = os.getenv("OPENAI_DEPLOYMENT_VERSION")
OPENAI_EMBEDDING_VERSION = os.getenv("OPENAI_EMBEDDING_VERSION")

#init Azure OpenAI
openai.api_type = "azure"
openai.api_version = OPENAI_DEPLOYMENT_VERSION
openai.api_base = OPENAI_DEPLOYMENT_ENDPOINT
openai.api_key = OPENAI_API_KEY

In [4]:
llm = OpenAI(engine="text-davinci-003", temperature=0)

WARNING! engine is not default parameter.
                    engine was transferred to model_kwargs.
                    Please confirm that engine is what you intended.


## Prompt Template

In [5]:
from langchain import PromptTemplate

prompt1 = """Who directed movie {movieName}"""

promptTemplate1 = PromptTemplate(
    input_variables = ["movieName"],
    template = prompt1
)

promptTemplate1.format(movieName = "RRR")

'Who directed movie RRR'

In [6]:
from langchain.prompts import load_prompt

loadedPrompt = load_prompt("./prompt.json")

loadedPrompt.format(movieName = "Titanic")

'Who directed movie Titanic'

## Chains

In [9]:
from langchain.chains import LLMChain, SequentialChain

movie_prompt = """Who directed movie {movieName}"""

movie_promptTemplate = PromptTemplate(
    input_variables = ["movieName"],
    template = movie_prompt
)

movie_chain = LLMChain(llm=llm, prompt=movie_promptTemplate, output_key="output")

director_prompt = """When was the director mentioned in the movie '{output}' born?"""

director_promptTemplate = PromptTemplate(
    input_variables=["output"],
    template=director_prompt,
)

director_chain = LLMChain(llm=llm, prompt=director_promptTemplate, output_key="result")

sequence = SequentialChain(
    chains = [movie_chain, director_chain],
    input_variables=["movieName"],
    output_variables=["output", "result"],
    verbose=True
)

response = sequence({
    "movieName": "RRR"
})

print(response.get("output"), response.get("result"))




> Entering new  chain...

> Finished chain.


The movie RRR is directed by S. S. Rajamouli. 

S. S. Rajamouli was born on October 10, 1973.


## Agents

In [10]:
from langchain.agents import initialize_agent, AgentType, Tool
from langchain.utilities import SerpAPIWrapper
from langchain.chains import LLMMathChain

search = SerpAPIWrapper(serpapi_api_key = os.getenv("SERPER_API_KEY"))
llm_math = LLMMathChain.from_llm(llm=llm)

tools = [
    Tool(
        name = "Search",
        func = search.run,
        description = '''useful for when you need to answer questions about current events,
            getting flights information like cheapest flights and best flights,
            getting weather forecast infomation like temperature and humidity
            '''
    ),
    Tool(
        name = 'Calculator',
        func = llm_math.run,
        description = 'Useful for when you need to answer questions about math.'
    )
]

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose = True
)

In [11]:
agent.run("How is the weather in New York? Convert its temperature from fahrenheit to celsius")



> Entering new  chain...
 I need to find the temperature in New York and then convert it
Action: Search
Action Input: Weather in New York
Observation: Sun 16 | Night ... Thunderstorms early, then partly cloudy after midnight. Potential for heavy rainfall. Low 73F. Winds light and variable. Chance of rain 60%.
Thought: I need to convert the temperature from fahrenheit to celsius
Action: Calculator
Action Input: 73F to Celsius
Observation: Answer: 22.77777777777778
Thought: I now know the final answer
Final Answer: The temperature in New York is 22.78°C.

> Finished chain.


'The temperature in New York is 22.78°C.'

## Memory

In [12]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()

conversation = ConversationChain(
    llm = llm,
    verbose = True,
    memory = memory
)

In [13]:
conversation.run("Hey! My name is Akash")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hey! My name is Akash
AI:

> Finished chain.


" Hi Akash, my name is AI. It's nice to meet you. What brings you here today?"

In [14]:
conversation.run("duh! what did I say my name was?")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hey! My name is Akash
AI:  Hi Akash, my name is AI. It's nice to meet you. What brings you here today?
Human: duh! what did I say my name was?
AI:

> Finished chain.


' You said your name is Akash. Is there something else I can help you with?'

## Callbacks

In [31]:
from langchain.callbacks.base import BaseCallbackHandler
from langchain.schema import LLMResult, HumanMessage
from typing import Any, Dict, List

callbackPrompt = """Who directed movie {movieName}"""

callbackPromptTemplate = PromptTemplate(
    input_variables = ["movieName"],
    template = prompt1
)

class MyCustomHandler(BaseCallbackHandler):
    count: int = 0
    def on_chain_start(self, serialized: Dict[str, Any], inputs: Dict[str, Any], **kwargs: Any) -> None:
        print('Entering new chain with inputs: ', inputs)
        self.count += 1

    def on_chain_end(self, token: str, **kwargs) -> None:
        print(f"Chain ended with: {token}")
        print(f"Completed chain: {self.count}")
    
    def on_llm_end(self, response: LLMResult, **kwargs: Any) -> Any:
        """Run when LLM ends running."""
        print(f"LLM ended with: {response}")

callbackChain = LLMChain(llm=llm, prompt=callbackPromptTemplate, verbose=True, callbacks=[MyCustomHandler()])

callbackChain.run("Titanic")

Entering new chain with inputs:  {'movieName': 'Titanic'}


> Entering new  chain...
Prompt after formatting:
Who directed movie Titanic
Chain ended with: {'text': '\n\nJames Cameron directed the movie Titanic.'}
Completed chain: 1

> Finished chain.


'\n\nJames Cameron directed the movie Titanic.'